In [1]:
import os
import numpy as np

In [2]:
basedir = "/home/timeTravelling/yuanke/nerf-pytorch/reconstruct_output"

In [3]:
poses_arr = np.load(os.path.join(basedir, 'poses_bounds.npy'))

In [4]:
poses = poses_arr[:, :-2].reshape([-1, 3, 5]).transpose([1,2,0])
poses.shape

(3, 5, 100)

In [5]:
poses = np.moveaxis(poses, -1, 0).astype(np.float32)

In [6]:
poses.shape

(100, 3, 5)

In [9]:
H, W, K, c2w =800,800,0,poses[0,:3,:4]

In [10]:
i, j = np.meshgrid(np.arange(W, dtype=np.float32), np.arange(H, dtype=np.float32), indexing='xy')

In [12]:
k = np.meshgrid(np.arange(W, dtype=np.float32), np.arange(H, dtype=np.float32), indexing='xy')

In [14]:
hwf = poses[0,:3,-1]
H, W, focal = hwf
H, W = int(H), int(W)
hwf = [H, W, focal]
K = np.array([
    [focal, 0, 0.5*W],
    [0, focal, 0.5*H],
    [0, 0, 1]
])

In [16]:
dirs = np.stack([(i-K[0][2])/K[0][0], -(j-K[1][2])/K[1][1], -np.ones_like(i)], -1)

In [25]:
rays_d = np.sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)

In [29]:
poses.shape

(100, 3, 5)

In [30]:
def get_rays_np(H, W, K, c2w):
    i, j = np.meshgrid(np.arange(W, dtype=np.float32), np.arange(H, dtype=np.float32), indexing='xy')
    dirs = np.stack([(i-K[0][2])/K[0][0], -(j-K[1][2])/K[1][1], -np.ones_like(i)], -1)
    rays_d = np.sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)  # dot product, equals to: [c2w.dot(dir) for dir in dirs]
    rays_o = np.broadcast_to(c2w[:3,-1], np.shape(rays_d))
    return rays_o, rays_d
rays = np.stack([get_rays_np(H, W, K, p) for p in poses[:,:3,:4]], 0)

In [31]:
rays.shape

(100, 2, 800, 800, 3)